In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from fit_lognormal_traces import fit_lognormal
from lognormal_templates import lognormal_signal
from names import DICT_COMP_COLORS, eMUON, eEM_HAD, eEM_MU, eEM_PURE, DICT_COMP_LABELS
from time_templates.utilities.plot import plot_profile_1d, plot_hist
from time_templates.utilities.fitting import plot_fit_curve
from lognormal_templates import get_m_s_lognormal_comp, lognormal_signal, lognormal_pdf

SAVE = True
from merge_mean_df import merge_means
from fit_lognormal_traces import get_trace_fit
if SAVE:
    plt.style.use('thesis')

In [ ]:
def make_mean(df):
    df_gb = df.groupby(['MCDXstation_bin', 'MCr_round'])
    df_mean = df_gb.mean()
    for key in df.keys():
        if 'trace' in key:
            sum_trace = df_gb[key].sum()
            sum_sq_trace = df_gb[key].apply(lambda x: np.sum(x**2, axis=0))
            n = df_gb[key].count()
            df_mean[key+"_mean"] = np.where(n > 1, sum_trace/np.maximum(n, 1), np.nan)
            df_mean[key+"_var"] = np.where(n > 1, sum_sq_trace/np.maximum(n, 1) - sum_trace**2/np.maximum(n, 1)**2, np.nan)
            df_mean['nstations'] = n
    return df_mean.dropna(subset='wcd_muon_trace_var')

df_p_19 = pd.read_pickle('/home/mart/auger/data/time_templates/binned_df/binned_df_EPOS_LHC_proton_19_19.5_phased_traces.pl')
df_p_195 = pd.read_pickle('/home/mart/auger/data/time_templates/binned_df/binned_df_EPOS_LHC_proton_19.5_20_phased_traces.pl')
df_Fe_19 = pd.read_pickle('/home/mart/auger/data/time_templates/binned_df/binned_df_EPOS_LHC_iron_19_19.5_phased_traces.pl')
df_Fe_195 = pd.read_pickle('/home/mart/auger/data/time_templates/binned_df/binned_df_EPOS_LHC_iron_19.5_20_phased_traces.pl')
df_p = pd.concat([df_p_19, df_p_195])
df_Fe = pd.concat([df_Fe_19, df_Fe_195])
# df = pd.concat([df_p_19, df_p_195, df_Fe_19, df_Fe_195])
# df = make_mean(df)
# df_p_19 = make_mean(df_p_19)
# df_p_195 = make_mean(df_p_195)
# df_Fe_19 = make_mean(df_Fe_19)
# df_Fe_195 = make_mean(df_Fe_195)
df_p = make_mean(df_p)
df_Fe = make_mean(df_Fe)

In [ ]:
def is_valid(m):
    if  (m.valid and m.accurate and not m.fmin.has_parameters_at_limit and m.fmin.has_posdef_covar):
        return True
    else:
        return False

def fit_traces(df):
    print("running")
    dd = defaultdict(list)
    trace_keys = [key[:-5] for key in df.keys() if "trace_mean" in key]
    min_stations = 10
    for (DX_bin, r), row in df.iterrows():
        dofit=True
        if row["nstations"] < min_stations:
            dofit = False
        d_fit = {}
        for key in trace_keys:
            success = True
            if dofit:
                try:
                    m = get_trace_fit(row, key, cdf_min=0.0, cdf_max=0.95, 
                                      fit_t0=True)
                    if not is_valid(m):
                        m.simplex()
                        m.migrad()
                    if not is_valid(m):
                        m = get_trace_fit(row, key, cdf_min=0.0, cdf_max=0.95, 
                                        fit_t0=True)
                        m.simplex()
                        m.migrad()

                    if not is_valid(m):
                        pfit = [np.nan, np.nan, np.nan]
                        perr = [np.nan, np.nan, np.nan]
                        red_chi2 = np.nan
                        success = False
                    else:
                        success = True
                        pfit = m.values
                        red_chi2 = m.fmin.reduced_chi2
                        perr = m.errors
                except RuntimeError:
                    pfit = [np.nan, np.nan, np.nan]
                    perr = [np.nan, np.nan, np.nan]
                    red_chi2 = np.nan
                    success = False
            else:
                success = False
                pfit = [np.nan, np.nan, np.nan]
                perr = [np.nan, np.nan, np.nan]
                red_chi2 = np.nan

            d_fit[key + "_mfit"] = pfit[0]
            d_fit[key + "_sfit"] = pfit[1]
            d_fit[key + "_merr"] = perr[0]
            d_fit[key + "_serr"] = perr[1]
            d_fit[key+ "_t0"] = pfit[2]
            d_fit[key+ "_t0err"] = perr[2]
            d_fit[key + "_redchi2"] = red_chi2
            d_fit[key + "_success"] = success

        for key in d_fit:
            dd[key].append(d_fit[key])

    df_fit = pd.DataFrame(dd)
    idx = df.index.names
    df_join = df.reset_index().join(df_fit)
    print("...done")
    return df_join.set_index(idx).swaplevel(0, 1)

# df_p_19 = fit_traces(df_p_19)
# df_p_195 = fit_traces(df_p_195)
# df_Fe_19 = fit_traces(df_Fe_19)
# df_Fe_195 = fit_traces(df_Fe_195)
# df = fit_traces(df)
df_p = fit_traces(df_p)
df_Fe = fit_traces(df_Fe)

In [ ]:
def DX_func(DX, a, b, c=0, d=0):
    DXref = DX/400 - 1
    return a + b*DXref + c*DXref**2 + d*DXref**3

def fit_func_bootstrap(x, m, merr, fitfunc=DX_func, p0=[1, 1, 1, 1], ax=None):
    _, (pfit, perr, chi2, ndof) = plot_fit_curve(x, m, yerr=merr, p0=p0, func=fitfunc, ax=None)
    mpred = fitfunc(x, *pfit)
    residual = np.abs((m-mpred)/merr)
    mask = (residual < 5)
    if len(x[mask]) < 4:
        return pfit, perr, chi2, ndof
    else:
        ax, (pfit, perr, chi2, ndof) = plot_fit_curve(x[mask], m[mask], yerr=merr[mask], p0=pfit, func=fitfunc, ax=ax, smoother_x=True)
    return pfit, perr, chi2, ndof

def parmaterize_DX(df):
    print('running...')
    f, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes = axes.flatten()
    f2, axes2 = plt.subplots(2, 2, figsize=(12, 12))
    axes2 = axes2.flatten()
    d_DX_fit = {}
    rs = np.sort(df.index.get_level_values(level=0).unique())
    nr = len(rs)
    NPARAMETERS = 4
    empty_array = np.empty((nr, NPARAMETERS))

    for comp, ax1, ax2 in zip(DICT_COMP_LABELS, axes, axes2):
        d_DX_fit[comp] = {}
        empty_array[:] = np.nan
        d_DX_fit[comp]['m'] = empty_array.copy()#.tolist()
        d_DX_fit[comp]['s'] = empty_array.copy()#.tolist()
        d_DX_fit[comp]['merr'] = empty_array.copy()#.tolist()
        d_DX_fit[comp]['serr'] = empty_array.copy()#.tolist()
        for i, r in enumerate(rs):
            df_ = df.loc[r]
            dx = df_['MCDXstation']
            red_chi2 = df_[f'wcd_{comp}_trace_redchi2']
            m = df_[f'wcd_{comp}_trace_mfit']
            merr = df_[f'wcd_{comp}_trace_merr'] #* np.sqrt(red_chi2)
            s = df_[f'wcd_{comp}_trace_sfit'] #* np.sqrt(red_chi2)
            serr = df_[f'wcd_{comp}_trace_serr']
            success = df_[f'wcd_{comp}_trace_success']
            t0 = df_[f'wcd_{comp}_trace_t0']
            mask = np.isfinite(dx*m*merr*s*serr) & (success) & (m < 8) & (m > 3) & (s > 0.2) & (s < 1.2)
            if len(dx[mask]) < 4:
                continue 
#             d_DX_fit[comp][r] = {}
            pfit, perr, chi2, ndof = fit_func_bootstrap(dx[mask], m[mask], merr[mask], ax=ax1, p0=[6, 0, 0, 0])
#             print(pfit)
            d_DX_fit[comp]['m'][i, :] = pfit
            d_DX_fit[comp]['merr'][i, :] = perr
#             d_DX_fit[comp][r]['m'] = pfit
#             d_DX_fit[comp][r]['merr'] = perr
            pfit, perr, chi2, ndof = fit_func_bootstrap(dx[mask], s[mask], serr[mask], ax=ax2, p0=[0.7, 0])  
            d_DX_fit[comp]['s'][i, :2] = pfit
            d_DX_fit[comp]['serr'][i, :2] = perr
            d_DX_fit[comp]['s'][i, 2:] = 0
            d_DX_fit[comp]['serr'][i, 2:] = 0
#             d_DX_fit[comp][r]['s'] = pfit
#             d_DX_fit[comp][r]['serr'] = perr
    print('...done')
    return d_DX_fit

In [ ]:
df = pd.read_pickle('/home/mart/auger/data/time_templates/lognormal_fit/df_fitted_lognormals.pl')

In [ ]:
def make_plot(compare_energy=False, compare_proton_iron=False, rs=[500, 1000, 1600]):

    f, axes = plt.subplots(2, 2, figsize=(6, 6))
    f2, axes2 = plt.subplots(2, 2, figsize=(6, 6))
    axes = axes.flatten()
    axes2 = axes2.flatten()
    x = np.linspace(0, 950)
    rall = np.sort(df.index.get_level_values(level=0).unique())
    for comp, ax, ax2 in zip(DICT_COMP_LABELS, axes, axes2):
        for r in rs:
            nstations = 10
            ir = np.argmin(r > rall)
#             df_ = df_p.loc[r].query(f'nstations > {nstations} & wcd_{comp}_trace_success == True')
            
#             pl = ax.errorbar(df_['MCDXstation'], df_[f'wcd_{comp}_trace_mfit'], yerr=df_[f'wcd_{comp}_trace_merr'],
#                              marker='o', ls='', label=f'$r = {r}$ m')
#             ax.plot(x, DX_func(x, *d_DX_fit_p_195[comp]['m'][ir]), color=pl[0].get_color())

#             pl = ax2.errorbar(df_['MCDXstation'], df_[f'wcd_{comp}_trace_sfit'], yerr=df_[f'wcd_{comp}_trace_serr'],
#                              marker='o', ls='', label=f'$r = {r}$ m')
# #             ax2.plot(x, DX_func(x, *d_DX_fit_p_195[comp]['s'][ir]), color=pl[0].get_color())

#             if compare_proton_iron:
#                 df_ = df_Fe.loc[r].query(f'nstations > {nstations}')
#                 ax.errorbar(df_['MCDXstation'], df_[f'wcd_{comp}_trace_mfit'], yerr=df_[f'wcd_{comp}_trace_merr'],
#                                  marker='v', ls='', color=pl[0].get_color())
# #                 ax.plot(x, DX_func(x, *d_DX_fit_Fe_195[comp]['m'][ir]), color=pl[0].get_color(), ls='--')
#                 ax2.errorbar(df_['MCDXstation'], df_[f'wcd_{comp}_trace_sfit'], yerr=df_[f'wcd_{comp}_trace_serr'],
#                                  marker='v', ls='', color=pl[0].get_color())
# #                 ax2.plot(x, DX_func(x, *d_DX_fit_Fe_19[comp]['s'][ir]), color=pl[0].get_color(), ls='--')
            
            df_ = df.loc[r].query(f'nstations > {nstations} & wcd_{comp}_trace_success == True')
        
            pl = ax.errorbar(df_['MCDXstation'], df_[f'wcd_{comp}_trace_mfit'], yerr=df_[f'wcd_{comp}_trace_merr'],
                         marker='s', ls='', label=f'$r = {r}$ m')
            pl = ax2.errorbar(df_['MCDXstation'], df_[f'wcd_{comp}_trace_sfit'], yerr=df_[f'wcd_{comp}_trace_serr'],
                             marker='s', ls='', label=f'$r = {r}$ m')
            
            ax.plot(x, [get_m_s_lognormal_comp(x_, r, comp)[0] for x_ in x], color=pl[0].get_color())
            ax2.plot(x, [get_m_s_lognormal_comp(x_, r, comp)[1] for x_ in x], color=pl[0].get_color())

            ax.set_ylim([3.7, 8])
            ax2.set_ylim([0.25, 1.1])
            
            ax.set_xlabel('$\Delta X \, [\\rm g/cm^2]$')
            ax2.set_xlabel('$\Delta X \, [\\rm g/cm^2]$')
            ax.set_ylabel('$m$')
            ax2.set_ylabel('$s$')
            ax.set_title(f"${DICT_COMP_LABELS[comp]}$")
            ax2.set_title(f"${DICT_COMP_LABELS[comp]}$")
            ax.set_xticks(np.arange(0, 1200, 200))
            ax2.set_xticks(np.arange(0, 1200, 200))
    axes[0].legend()
    axes2[0].legend()
    f.subplots_adjust(wspace=0.25, hspace=0.4)
    f2.subplots_adjust(wspace=0.25, hspace=0.4)
    if SAVE:
        f.savefig("/home/mart/auger/thesis/thesis_overleaf/figs/chapter5/m_vs_DX.pdf")
        f2.savefig("/home/mart/auger/thesis/thesis_overleaf/figs/chapter5/s_vs_DX.pdf")

make_plot(compare_proton_iron=True) #compare_energy=True)

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(6, 3.8))
f2, axes2 = plt.subplots(2, 2, figsize=(6, 3.8))
axes = axes.flatten()
axes2 = axes2.flatten()
rs = [500, 1000, 1600]
rall = np.sort(df_p.index.get_level_values(level=0).unique())

colors = ['C0', 'C1', 'C2']
for comp, ax, ax2 in zip(DICT_COMP_LABELS, axes, axes2):
    for i, r in enumerate(rs):
        nstations = 10
        ir = np.argmin(r > rall)
        for df_, marker in zip([df_p, df_Fe], ['o', '^']):
            df_ = df_.loc[r].query(f'nstations > {nstations} & wcd_{comp}_trace_success == True')
            x = df_['MCDXstation'].values
            m = [get_m_s_lognormal_comp(x_, r, comp)[0] for x_ in x]
            s = [get_m_s_lognormal_comp(x_, r, comp)[1] for x_ in x]

            ax.errorbar(df_['MCDXstation'], df_[f'wcd_{comp}_trace_mfit']/m+i-1, yerr=df_[f'wcd_{comp}_trace_merr']/m,
                         marker=marker, ls='', label=f'$r = {r}$ m', color=colors[i])
            ax2.errorbar(df_['MCDXstation'], df_[f'wcd_{comp}_trace_sfit']/s+i-1, yerr=df_[f'wcd_{comp}_trace_serr']/s,
                 marker=marker, ls='', label=f'$r = {r}$ m', color=colors[i])
            ax.axhline(i, color='k', ls='--')
            ax2.axhline(i, color='k', ls='--')
            ax.set_ylim([-1, 3])
            ax2.set_ylim([-1, 3])
        ax.annotate(f'$r = {r}$ m', xy=(50, i+0.3))
        ax2.annotate(f'$r = {r}$ m', xy=(50, i+0.3))

    ax.set_xlabel('$\Delta X \, [\\rm g/cm^2]$')
    ax2.set_xlabel('$\Delta X \, [\\rm g/cm^2]$')
    ax.set_ylabel('$m/\hat{m} - 1$')
    ax2.set_ylabel('$s/\hat{s} -1$')
    ax.set_title(f"${DICT_COMP_LABELS[comp]}$")
    ax2.set_title(f"${DICT_COMP_LABELS[comp]}$")
    ax.set_xticks(np.arange(0, 1200, 200))
    ax2.set_xticks(np.arange(0, 1200, 200))
    f.subplots_adjust(wspace=0.25, hspace=0.55)
    f2.subplots_adjust(wspace=0.25, hspace=0.55)
            
if SAVE:
    f.savefig("/home/mart/auger/thesis/thesis_overleaf/figs/appendix//dev_m_vs_DX.pdf")
    f2.savefig("/home/mart/auger/thesis/thesis_overleaf/figs/appendix/dev_s_vs_DX.pdf")


In [ ]:
365/7

In [ ]:
300 / (365*4)

In [ ]:
#cpu time at stoomboot
y3 = 19372539
y4 = 22854936

dy = y4-y3
y2 = y3#-dy
y1 = y2#-dy
total = y1+y2+y3+y4
watt = 100

print(total/3600 * watt/1000, 'kwh')

In [ ]:
#laptop usage approx
watt = 20 #somewhere between 10-20
hours = 8 * 5 * 52 * 4
print(watt / 1000 * hours, 'kwh')

In [ ]:
df = pd.read_pickle('/home/mart/auger/data/time_templates/binned_df/binned_df_EPOS_LHC_proton_19_19.5_phased_traces.pl')

In [ ]:
ct2 = 0.85
DX = 220
r = 800
cp = 1
df_row = df.loc[ct2, DX, r, cp]

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(13, 12), sharey=True, sharex=True)
axes = axes.flatten()
t = np.arange(0, 600*25/3, 25/3)

REBIN = False
if REBIN:
    t = t.reshape((200, 3)).mean(axis=-1)
    det = 'UB_WCD'
else:
    det = 'UUB_WCD'

#Errorbary every 50ns and inflated by 2
DX = df_row['MCDXstation'].mean()
theta = df_row['MCTheta'].mean()
for comp, ax in zip([eMUON, eEM_PURE, eEM_MU, eEM_HAD], axes):
    traces = np.vstack(df_row[f'wcd_{comp}_trace'].values)

    n = len(traces)
    y = np.mean(traces, axis=0)
    median = np.median(traces, axis=0)
    yerr = np.std(traces, axis=0)/np.sqrt(n)*2
    if REBIN:
        y = y.reshape((200, 3)).mean(axis=-1)  # WARNING hardcoded
        yerr = np.sqrt(np.sum(yerr.reshape((200, 3)) ** 2, axis=-1)/3)
#     if comp == eMUON:
#         t0 = df_row['t0_wrt_pft_Rc_fit'].mean()
#         tmu = np.linspace(t0, t0+599*25/3, 600)
#         muon_dsdt = MuondSdt(r, np.arccos(cp), np.arccos(np.sqrt(theta)))
#         m, _, _ = fit_Xmumax_lam(tmu, y, yerr, muon_dsdt, p0=[550, 40], fix_lambda=False)
#         ax.plot(t, muon_dsdt.ds_dt_wcd(tmu), '--', color=DICT_COMP_COLORS[comp])
    minuit, imin, imax = fit_lognormal(t, y, yerr, cdf_min=0.0, cdf_max=0.95, det='UUB_WCD')
    m, s = minuit.values['m'], minuit.values['s']

#     m, s = v_get_m_s_lognormal_comp(DX, r, comp)
    ax.plot(t, traces[np.random.randint(0, n)], color='k', alpha=0.2, drawstyle='steps')
    ax.plot(t, lognormal_signal(t, m, s, det=det), color=DICT_COMP_COLORS[comp], ls='-', lw=1, label=f"${DICT_COMP_LABELS[comp]}$")
    ax.errorbar(t[::3], y[::3], 2*yerr[::3], color=DICT_COMP_COLORS[comp], ls='', marker='.', errorevery=1, mfc='none')
    ax.set_xlim([-25, 2500])
    ax.set_ylim([-1e-4, 0.0035])
    ax.legend(loc=2)
    ax_in = ax.inset_axes([0.53, 0.47, 0.45, 0.45])
    ax_in.plot(t, traces[np.random.randint(0, n)], color='k', alpha=0.2, drawstyle='steps')
    ax_in.plot(t, lognormal_signal(t, m, s, det=det), color=DICT_COMP_COLORS[comp], ls='-', lw=1)
#     if comp == eMUON:
#         ax_in.plot(t, muon_dsdt.ds_dt_wcd(tmu), '--', color=DICT_COMP_COLORS[comp])
    ax_in.errorbar(t[::3], y[::3], 2*yerr[::3], color=DICT_COMP_COLORS[comp], ls='', marker='.', errorevery=1, mfc='none')
    ax_in.set_xscale('log')
    ax_in.set_yscale('log')
    ax_in.set_ylim([1e-6, 0.0035])
    ax_in.tick_params(axis='both', labelsize=8)
    ax_in.minorticks_off()
#     ax.annotate(xy=(0.1, 0.95), text=f"${DICT_COMP_LABELS[comp]}$", xycoords='axes fraction', va='top')
#     ax.yaxis.set_tick_params(labelleft=True)
        
axes[2].set_xlabel('$t-t_0$ [ns]')
axes[3].set_xlabel('$t-t_0$ [ns]')
axes[0].set_ylabel('$\\dfrac{1}{|S|}\\dfrac{dS}{dt} [\\mathrm{ns^-1}]$')
axes[2].set_ylabel('$\\dfrac{1}{|S|}\\dfrac{dS}{dt} [\\mathrm{ns^-1}]$')
# axes[3].xaxis.set_tick_params(labelbottom=False)
# axes[0].yaxis.set_tick_params(labelleft=False)

f.subplots_adjust(wspace=0.06, hspace=0.06)
# if SAVE:
    plt.savefig('/home/mart/auger/thesis/thesis_overleaf/figs/chapter5/example_lognormal_fit.pdf', bbox_inches='tight')